In [1]:
import pandas as pd
import numpy as np

#load file
prices = pd.read_csv("prices.csv")
sales = pd.read_csv("sales.csv")

In [2]:
#convert ordered_at, updated_at to datetime object (using pd.to_datetime)
prices["updated_at"] = pd.to_datetime(prices["updated_at"])
sales["ordered_at"] = pd.to_datetime(sales["ordered_at"])

In [3]:
sales = sales.sort_values(by='ordered_at')
sales

,product_id,ordered_at,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1
...,...,...,...
67,4085861,2018-09-18 20:23:00,1
77,4085861,2018-09-18 20:43:00,1
79,4085861,2018-09-18 20:54:00,1
87,3954203,2018-09-18 21:26:00,1


In [4]:
prices = prices.sort_values(by='updated_at')
prices

,product_id,old_price,new_price,updated_at
4,3954203,68800,60000,2018-09-10 16:32:00
7,3998909,19000,17000,2018-09-10 16:35:00
0,64,270000,239000,2018-09-10 16:37:00
11,4085861,60000,53500,2018-09-11 08:51:00
1,3954203,60000,64000,2018-09-11 11:54:00
9,4085861,53500,67000,2018-09-12 03:51:00
6,3998909,17000,15500,2018-09-13 06:43:00
13,4085861,67000,62500,2018-09-13 06:43:00
3,3954203,64000,60500,2018-09-15 03:49:00
12,4085861,62500,58000,2018-09-15 03:51:00


In [5]:
# merge sales and prices table with direction='backward' (default value, which can be optionally omitted)
merge_backward = pd.merge_asof(sales, prices, by='product_id', left_on='ordered_at', right_on='updated_at', direction='backward')
merge_backward = merge_backward[['product_id', 'ordered_at', 'quantity_ordered', 'new_price']].rename(columns={'new_price':'price'})
merge_backward.head(10)

,product_id,ordered_at,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,NaN
2,4085861,2018-09-11 06:53:00,1,NaN
3,4085861,2018-09-11 08:24:00,1,NaN
4,4085861,2018-09-11 09:30:00,1,53500.0
5,4085861,2018-09-11 11:06:00,1,53500.0
6,3954203,2018-09-11 11:11:00,1,60000.0
7,3954203,2018-09-11 11:11:00,1,60000.0
8,4085861,2018-09-11 11:34:00,1,53500.0
9,4085861,2018-09-11 11:47:00,2,53500.0


In [6]:
# merge merge_backward and prices tables with direction='forward'
merge_forward = pd.merge_asof(merge_backward, prices, by='product_id', left_on='ordered_at', right_on='updated_at', direction='forward')
merge_forward.head()

,product_id,ordered_at,quantity_ordered,price,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,60000.0,60000.0,64000.0,2018-09-11 11:54:00
1,4085861,2018-09-11 06:26:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,53500.0,53500.0,67000.0,2018-09-12 03:51:00


In [7]:
# fill null values in 'price' column by 'old_price' column where necessary
merge_forward['price'] = merge_forward['price'].fillna(merge_forward['old_price'])
final_data = merge_forward[['product_id', 'ordered_at', 'quantity_ordered', 'price']]
final_data

,product_id,ordered_at,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,60000.0
2,4085861,2018-09-11 06:53:00,1,60000.0
3,4085861,2018-09-11 08:24:00,1,60000.0
4,4085861,2018-09-11 09:30:00,1,53500.0
...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,52000.0
171,4085861,2018-09-18 20:43:00,1,52000.0
172,4085861,2018-09-18 20:54:00,1,52000.0
173,3954203,2018-09-18 21:26:00,1,57500.0


In [8]:
final_data['revenue'] = final_data['quantity_ordered'] * final_data['price']
revenue_by_product_and_price = final_data.groupby(['product_id', 'price'], as_index=False)['revenue'].sum()
revenue_by_product_and_price

C:\Users\40 MINH KHAI\AppData\Local\Temp\ipykernel_20808\2432931725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['revenue'] = final_data['quantity_ordered'] * final_data['price']


,product_id,price,revenue
0,64,239000.0,956000.0
1,3954203,57500.0,57500.0
2,3954203,60000.0,180000.0
3,3954203,64000.0,640000.0
4,3998909,15500.0,15500.0
5,3998909,16500.0,231000.0
6,3998909,17000.0,34000.0
7,4085861,52000.0,1040000.0
8,4085861,53500.0,2140000.0
9,4085861,58000.0,2204000.0


In [9]:
total_revenue = final_data.groupby('product_id', as_index=False)['revenue'].sum()
total_revenue

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0
